## Problem Statement

### Business Context

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

**1. Diagnostic Assistance**: "What are the common symptoms and treatments for pulmonary embolism?"

**2. Drug Information**: "Can you provide the trade names of medications used for treating hypertension?"

**3. Treatment Plans**: "What are the first-line options and alternatives for managing rheumatoid arthritis?"

**4. Specialty Knowledge**: "What are the diagnostic steps for suspected endocrine disorders?"

**5. Critical Care Protocols**: "What is the protocol for managing sepsis in a critical care unit?"

### Objective

As an AI specialist, your task is to develop a RAG-based AI solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

## Installing and Importing Necessary Libraries and Dependencies

In [ ]:
# Installation for GPU llama-cpp-python
# uncomment and run the following code in case GPU is being used
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1
!pip install llama-cpp-python

# Installation for CPU llama-cpp-python
# uncomment and run the following code in case GPU is not being used
# !CMAKE_ARGS="-DLLAMA_CUBLAS=off" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 MB 32.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.2 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.11-cp311-cp311-linux_x86_64.whl size=4122682 sha256=2cdb7628179e5c7da6022165f9cd430a095c74727432e41b83f29daeca9689e5
  Stored in directory: /root/.cache/pip/wheels/1a/65/e9/3bd26ec174c6e148cce8a3876d7ed32652e3508ebe262c197a
Successfully built llama-cpp-python


In [ ]:
!pip install numpy
!pip install pandas

In [ ]:
# For installing the libraries & downloading models from HF Hub
%pip install huggingface_hub
%pip install tiktoken
%pip install pymupdf
%pip install langchain
%pip install langchain-community
%pip install chromadb
%pip install sentence-transformers



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 120.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.4 MB/s eta 0:00:

In [ ]:
#Libraries for downloading and loading the llm
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

## Question Answering using LLM

#### Downloading and Loading the model

In [ ]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"


model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

print(f"Model downloaded to: {model_path}")

llm = Llama(
    model_path=model_path,
    n_ctx=2300,
    n_gpu_layers=8,
    n_batch=128
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.2.Q6_K.gguf:   0%|          | 0.00/5.94G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loade

Model downloaded to: /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q6_K.gguf


load_tensors:   CPU_Mapped model buffer size =  5666.09 MiB
...................................................................................................
llama_context: constructing llama_context
llama_context: n_seq_max     = 1
llama_context: n_ctx         = 2300
llama_context: n_ctx_per_seq = 2300
llama_context: n_batch       = 128
llama_context: n_ubatch      = 128
llama_context: causal_attn   = 1
llama_context: flash_attn    = 0
llama_context: freq_base     = 1000000.0
llama_context: freq_scale    = 1
llama_context: n_ctx_per_seq (2300) < n_ctx_train (32768) -- the full capacity of the model will not be utilized
set_abort_callback: call
llama_context:        CPU  output buffer size =     0.12 MiB
create_memory: n_ctx = 2304 (padded)
llama_kv_cache_unified: layer   0: dev = CPU
llama_kv_cache_unified: layer   1: dev = CPU
llama_kv_cache_unified: layer   2: dev = CPU
llama_kv_cache_unified: layer   3: dev = CPU
llama_kv_cache_unified: layer   4: dev = CPU
llama_kv_cache_unified

#### Response

In [ ]:
# Temparature is kept at 0 because for this business use case we want factual answers

def response(query,max_tokens=200,temperature=0,top_p=0.95,top_k=50):
    model_output = llm(
      prompt=query,
      max_tokens=max_tokens,
      temperature=temperature,
      top_p=top_p,
      top_k=top_k
    )

    return model_output['choices'][0]['text']

### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
query1 = "What is the protocol for managing sepsis in a critical care unit?"
print(response(query1))


llama_perf_context_print:        load time =    1484.88 ms
llama_perf_context_print: prompt eval time =    1484.31 ms /    16 tokens (   92.77 ms per token,    10.78 tokens per second)
llama_perf_context_print:        eval time =   31809.02 ms /   199 runs   (  159.84 ms per token,     6.26 tokens per second)
llama_perf_context_print:       total time =   33414.93 ms /   215 tokens




Sepsis is a life-threatening condition that can arise from an infection, and it requires prompt recognition and aggressive management in a critical care unit. The following are the general steps for managing sepsis in a critical care unit:

1. Early recognition and suspicion: Septic patients may present with non-specific symptoms such as fever, chills, tachycardia, tachypnea, altered mental status, or lactic acidosis. It is essential to have a high index of suspicion for sepsis, especially in patients with known infections or risk factors.
2. Initial assessment and resuscitation: The first step in managing sepsis is to assess and resuscitate the patient. This includes assessing airway, breathing, circulation, and disability (ABCD) and providing appropriate interventions such as oxygen therapy, fluid resuscitation, and vasopressor support if necessary.
3. Source control:


### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
query2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
print(response(query2))

Llama.generate: 2 prefix-match hit, remaining 32 prompt tokens to eval
llama_perf_context_print:        load time =    1484.88 ms
llama_perf_context_print: prompt eval time =    3190.47 ms /    32 tokens (   99.70 ms per token,    10.03 tokens per second)
llama_perf_context_print:        eval time =   32822.60 ms /   199 runs   (  164.94 ms per token,     6.06 tokens per second)
llama_perf_context_print:       total time =   36132.31 ms /   231 tokens




Appendicitis is a medical condition characterized by inflammation of the appendix, a small pouch that extends from the cecum, the first part of the large intestine. The symptoms of appendicitis can vary from person to person, but the following are the most common:

1. Abdominal pain: The pain is typically located in the lower right side of the abdomen and may be dull at first, but it can quickly become sharp and severe. The pain may worsen when you move, cough, or sneeze.
2. Loss of appetite: You may lose your appetite due to the abdominal pain or feel nauseous.
3. Nausea and vomiting: You may feel sick to your stomach and vomit.
4. Fever: A fever of 100.4°F (38°C) or higher is common with appendicitis.
5. Const


### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
query3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
print(response(query3))

Llama.generate: 4 prefix-match hit, remaining 34 prompt tokens to eval
llama_perf_context_print:        load time =    1484.88 ms
llama_perf_context_print: prompt eval time =    2447.22 ms /    34 tokens (   71.98 ms per token,    13.89 tokens per second)
llama_perf_context_print:        eval time =   32804.30 ms /   199 runs   (  164.85 ms per token,     6.07 tokens per second)
llama_perf_context_print:       total time =   35372.69 ms /   233 tokens




Sudden patchy hair loss, also known as alopecia areata, is a common autoimmune disorder that affects the hair follicles, leading to hair loss in small, round patches on the scalp, beard, or other areas of the body. The exact cause of alopecia areata is not known, but it is believed to be related to a problem with the immune system.

There are several treatments that have been shown to be effective in addressing sudden patchy hair loss:

1. Corticosteroids: Corticosteroids are anti-inflammatory medications that can help reduce inflammation and suppress the immune system, allowing the hair follicles to regrow. They can be applied topically or taken orally.
2. Minoxidil: Minoxidil is a medication that has been shown to promote hair growth in some people with alopecia areata. It is applied


### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
query4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
print(response(query4))

Llama.generate: 2 prefix-match hit, remaining 28 prompt tokens to eval
llama_perf_context_print:        load time =    1484.88 ms
llama_perf_context_print: prompt eval time =    2003.62 ms /    28 tokens (   71.56 ms per token,    13.97 tokens per second)
llama_perf_context_print:        eval time =   32784.29 ms /   199 runs   (  164.75 ms per token,     6.07 tokens per second)
llama_perf_context_print:       total time =   34911.69 ms /   227 tokens




A person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function, is typically diagnosed with a traumatic brain injury (TBI). The treatment for a TBI depends on the severity and location of the injury, as well as the individual's overall health and age.

Immediate treatment for a TBI may include:

1. Emergency medical care: This may include surgery to remove hematomas or other obstructions, as well as treatment for other injuries.
2. Medications: Depending on the symptoms, medications may be prescribed to manage pain, reduce swelling, prevent or treat infections, or control seizures.
3. Rehabilitation: Rehabilitation may include physical therapy, occupational therapy, speech therapy, and cognitive rehabilitation to help the person regain lost skills and functions.
4. Supportive care: This may include assistance with daily activities


### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
query5 = "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
print(response(query5))

Llama.generate: 2 prefix-match hit, remaining 35 prompt tokens to eval
llama_perf_context_print:        load time =    1484.88 ms
llama_perf_context_print: prompt eval time =    2505.09 ms /    35 tokens (   71.57 ms per token,    13.97 tokens per second)
llama_perf_context_print:        eval time =   32807.68 ms /   199 runs   (  164.86 ms per token,     6.07 tokens per second)
llama_perf_context_print:       total time =   35434.18 ms /   234 tokens




First and foremost, if a person has fractured their leg during a hiking trip, it is essential to ensure their safety and prevent further injury. Here are some necessary precautions and treatment steps:

1. Assess the situation: Check the extent of the injury and assess the person's condition. If the fracture is open or the person is in severe pain, immobilize the leg with a splint or a makeshift sling to prevent any movement.
2. Call for help: If possible, call for emergency medical assistance. If there is no cell phone reception, try to signal for help using a mirror, whistle, or other means.
3. Provide first aid: Apply a sterile dressing to the injury to prevent infection. If the fracture is open, apply pressure to stop any bleeding.
4. Immobilize the leg: Use a splint, a makeshift sling, or a tour


Response Quality: The LLM provides generally relevant and factually correct answers to medical queries, but the depth and specificity can vary. Some answers are incomplete or stop mid-sentence, especially for complex or multi-part questions.

Consistency: There is occasional inconsistency in the level of detail. For example, some answers list only a few symptoms or steps, while others are more comprehensive.

Limitations:

The LLM sometimes omits critical context or nuances, such as when to escalate care or specific contraindications.

Answers may lack explicit references to authoritative sources, which is important in medical settings.

There is a risk of hallucination or overgeneralization, especially for less common or ambiguous queries.

## Question Answering using LLM with Prompt Engineering

In [ ]:
# Add instructions to the prompt for better response generation
# Given the sensitivity of medical diagnosis, I am adding a prompt to minimize hallucination
system_prompt = "You are a helpful and knowledgeable medical assistant. Answer the following medical question accurately and concisely based on common medical knowledge. If you don't know the answer, please state that you don't have enough information."
print ("System Prompt:" + system_prompt)

System Prompt:You are a helpful and knowledgeable medical assistant. Answer the following medical question accurately and concisely based on common medical knowledge. If you don't know the answer, please state that you don't have enough information.


### Query 1: What is the protocol for managing sepsis in a critical care unit?

In [ ]:
query1 = system_prompt + "\n" + "What is the protocol for managing sepsis in a critical care unit?"
print(response(query1))


Llama.generate: 1 prefix-match hit, remaining 61 prompt tokens to eval
llama_perf_context_print:        load time =    1484.88 ms
llama_perf_context_print: prompt eval time =    4299.52 ms /    61 tokens (   70.48 ms per token,    14.19 tokens per second)
llama_perf_context_print:        eval time =   32964.31 ms /   199 runs   (  165.65 ms per token,     6.04 tokens per second)
llama_perf_context_print:       total time =   37388.56 ms /   260 tokens




Sepsis is a life-threatening condition caused by a severe infection. In a critical care unit, managing sepsis involves the following steps:

1. Early recognition and diagnosis: Monitor vital signs, laboratory values, and clinical symptoms closely. Suspect sepsis in any patient with suspected or confirmed infection and organ dysfunction.
2. Immediate fluid resuscitation: Administer intravenous fluids to maintain adequate blood pressure and organ perfusion. The goal is to achieve a mean arterial pressure (MAP) of 65 mmHg or higher.
3. Antibiotics: Administer broad-spectrum antibiotics as soon as possible based on the suspected infection site and microbiological culture results.
4. Source control: Identify and address the source of infection, such as removing an infected catheter or draining an abscess.
5. Vasopressors: If fluid res


### Query 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [ ]:
query2 = system_prompt + "\n" + "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"
print(response(query2))

Llama.generate: 48 prefix-match hit, remaining 32 prompt tokens to eval
llama_perf_context_print:        load time =    1484.88 ms
llama_perf_context_print: prompt eval time =    2346.33 ms /    32 tokens (   73.32 ms per token,    13.64 tokens per second)
llama_perf_context_print:        eval time =   32927.11 ms /   199 runs   (  165.46 ms per token,     6.04 tokens per second)
llama_perf_context_print:       total time =   35394.40 ms /   231 tokens




Appendicitis is a common inflammatory condition of the appendix, a small pouch located in the lower right side of the abdomen. The symptoms of appendicitis can include:

1. Sudden pain in the lower right abdomen, which may start as a mild ache and gradually develop into a sharp pain.
2. Loss of appetite and feeling sick to your stomach (nausea).
3. Fever, which may be low-grade at first but can rise as high as 103°F (39.4°C).
4. Vomiting.
5. Constipation or diarrhea.
6. Abdominal swelling and tenderness.
7. Inability to pass gas or have a bowel movement.
8. Pain in the lower back, on the right side.
9. Feeling restless or unable to find a comfortable position.


### Query 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [ ]:
query3 = system_prompt + "\n" + "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"
print(response(query3))

Llama.generate: 50 prefix-match hit, remaining 34 prompt tokens to eval
llama_perf_context_print:        load time =    1484.88 ms
llama_perf_context_print: prompt eval time =    2478.53 ms /    34 tokens (   72.90 ms per token,    13.72 tokens per second)
llama_perf_context_print:        eval time =   32968.36 ms /   199 runs   (  165.67 ms per token,     6.04 tokens per second)
llama_perf_context_print:       total time =   35569.89 ms /   233 tokens




Sudden patchy hair loss, also known as alopecia areata, is an autoimmune condition that causes hair loss in small, round patches on the scalp, beard, or other areas of the body. The exact cause is unknown, but it's believed to be related to a problem with the immune system.

Effective treatments for addressing sudden patchy hair loss include:

1. Corticosteroids: These are anti-inflammatory medications that can help reduce inflammation and suppress the immune system response. They can be applied topically or taken orally.
2. Immunotherapy: This involves the use of medications that stimulate the immune system to attack the hair loss. One such medication is minoxidil.
3. Hair transplantation: This is a surgical procedure in which healthy hair is transplanted from one area of the scalp to another. It's usually considered a


### Query 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [ ]:
query4 = system_prompt + "\n" + "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"
print(response(query4))

Llama.generate: 48 prefix-match hit, remaining 28 prompt tokens to eval
llama_perf_context_print:        load time =    1484.88 ms
llama_perf_context_print: prompt eval time =    2329.08 ms /    28 tokens (   83.18 ms per token,    12.02 tokens per second)
llama_perf_context_print:        eval time =   32839.01 ms /   199 runs   (  165.02 ms per token,     6.06 tokens per second)
llama_perf_context_print:       total time =   35290.27 ms /   227 tokens




For a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function, the recommended treatments depend on the specific type and severity of the injury. Here are some common treatments:

1. Emergency care: For severe brain injuries, the first priority is to provide emergency care to stabilize the patient's vital signs and prevent further damage. This may include surgery to remove hematomas or other obstructions, administering oxygen, and controlling seizures.
2. Medications: Depending on the symptoms, medications may be prescribed to manage various conditions such as pain, seizures, infections, and depression.
3. Rehabilitation: Rehabilitation is an essential part of treatment for brain injuries. It may include physical therapy to help the patient regain mobility, occupational therapy to help with daily living activities, speech therapy to improve communication skills, and cognitive therapy to help with memory and problem-

### Query 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

In [ ]:
query5 = system_prompt + "\n" + "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
print(response(query5))

Llama.generate: 48 prefix-match hit, remaining 35 prompt tokens to eval
llama_perf_context_print:        load time =    1484.88 ms
llama_perf_context_print: prompt eval time =    3638.63 ms /    35 tokens (  103.96 ms per token,     9.62 tokens per second)
llama_perf_context_print:        eval time =   32906.03 ms /   199 runs   (  165.36 ms per token,     6.05 tokens per second)
llama_perf_context_print:       total time =   36666.21 ms /   234 tokens




A leg fracture during a hiking trip requires prompt medical attention. Here are the necessary precautions and treatment steps:

1. Immobilize the fracture: Use a splint, sling, or a brace to prevent any movement of the affected leg. This will help reduce pain, prevent further damage, and promote proper healing.

2. Control bleeding: Apply direct pressure to the wound with a clean cloth to control any bleeding. Elevate the injured leg above heart level to help reduce swelling and bleeding.

3. Seek medical help: If the fracture is severe or if there are signs of shock (pale skin, rapid heartbeat, rapid breathing, or loss of consciousness), call for emergency medical assistance immediately.

4. Pain management: Use over-the-counter pain relievers, such as acetaminophen or ibuprofen, to help manage pain. Your healthcare provider may also prescribe


Response Quality: Prompt engineering (adding a system prompt to instruct the LLM to be concise, factual, and to admit uncertainty) improves the reliability and clarity of answers.

Groundedness: The responses are more focused, with less speculation and fewer unsupported claims. The LLM is more likely to state when it lacks sufficient information.

Structure: Answers are better organized, often using lists or stepwise instructions, which enhances readability for clinical users.

Limitations:

While hallucinations are reduced, the LLM still relies on its training data and may not always reflect the most current or authoritative medical guidelines.

The model does not cite specific sources, which can be a drawback for clinical auditability.

## Data Preparation for RAG

### Loading the Data

In [ ]:
## Data Preparation for RAG
### Loading the Data
#Libraries for processing dataframes, text
import json, os
import tiktoken
import pandas as pd

#Libraries for Loading Data, Chunking, Embedding, and Vector Databases
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

### Data Overview

In [ ]:
## Data Overview


pdf_path = "/content/medical_diagnosis_manual.pdf"
if not os.path.exists(pdf_path):
    raise FileNotFoundError(f"PDF not found at {pdf_path}")

# Use the PyMuPDFLoader to load the document
try:
    # Note: Loading large PDFs (>4,000 pages) may require significant memory. Consider chunked processing if RAM is limited.
    loader = PyMuPDFLoader(pdf_path)
    documents = loader.load()
    print(f"Loaded {len(documents)} pages from the PDF.")
except Exception as e:
    print(f"Error loading PDF: {e}")
    raise

Loaded 4114 pages from the PDF.


#### Checking the first 5 pages

In [ ]:
# Preview first 5 pages (or fewer if PDF is smaller) for debugging
for i in range(min(5, len(documents))):
    print(f"--- Page {i+1} ---")
    print(documents[i].page_content[:500] + "...")

#### Checking the number of pages
print(f"Number of pages: {len(documents)}")

--- Page 1 ---
anuguthalasandeepkumar@gmail.com
KUDXERABSF
personal use by anuguthalasandeepkum
shing the contents in part or full is liable...
--- Page 2 ---
anuguthalasandeepkumar@gmail.com
KUDXERABSF
This file is meant for personal use by anuguthalasandeepkumar@gmail.com only.
Sharing or publishing the contents in part or full is liable for legal action....
--- Page 3 ---
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    ....................................
--- Page 4 ---
491
Chapter 44. Foot & Ankle Disorders    .............................................................

#### Checking the number of pages

### Data Chunking

In [ ]:
## Function to do data chunking

def get_data_chunks(
    data,
    chunk_size=1000,
    chunk_overlap=200,
    split_method="recursive",
    separators=["\n\n", "\n", ". ", " ", ""],
    min_chunk_size=50,
    respect_sentence_boundaries=True,
    respect_paragraph_boundaries=True,
    length_function=len,
    max_chunks=None,
    add_metadata=True
):
    """
    Splits a list of documents into smaller chunks using a specified method.

    Args:
        data: A list of document objects (e.g., from Langchain loaders).
        chunk_size: The maximum size of each chunk.
        chunk_overlap: The number of characters to overlap between chunks.
        split_method: The method to use for splitting ('recursive').
        separators: A list of separators to use for splitting.
        min_chunk_size: The minimum size of each chunk.
        respect_sentence_boundaries: Whether to try to split on sentence boundaries.
        respect_paragraph_boundaries: Whether to try to split on paragraph boundaries.
        length_function: The function to use to measure chunk length.
        max_chunks: The maximum number of chunks to generate.
        add_metadata: Whether to add metadata to the chunks.

    Returns:
        A list of chunked documents.
    """
    try:
        if split_method == "recursive":
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size,
                chunk_overlap=chunk_overlap,
                length_function=length_function,
                is_separator_regex=False,
                separators=separators
            )
            chunks = text_splitter.split_documents(data)
        else:
            raise ValueError(f"Unsupported split_method: {split_method}")
    except Exception as e:
        print(f"Error during chunking: {e}")
        raise
    chunks = [chunk for chunk in chunks if length_function(chunk.page_content) >= min_chunk_size]
    if add_metadata:
        for chunk in chunks:
            if not chunk.metadata:
                chunk.metadata = {"source": "medical_diagnosis_manual.pdf", "page": 0}  # Fallback
    if max_chunks is not None:
        chunks = chunks[:max_chunks]
    return chunks

In [ ]:

# Utilize the get_data_chunks function for the loaded PDF
chunk_size = 500  # chunk size
chunk_overlap = 100 # chunk overlap

chunks = get_data_chunks(
    documents,
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    split_method="recursive",
    separators=["\n\n", "\n", ". ", " ", ""],
    min_chunk_size=50,
    respect_sentence_boundaries=True,
    respect_paragraph_boundaries=True,
    length_function=len,
    max_chunks=None, # Process all chunks
    add_metadata=True
)

# Print some validation and check statements
print(f"\n--- Chunking Validation ---")
print(f"Number of documents loaded: {len(documents)}")
print(f"Number of chunks created: {len(chunks)}")

# Check the first few chunks
print(f"\nFirst {min(5, len(chunks))} chunks:")
for i in range(min(5, len(chunks))):
  print(f"--- Chunk {i+1} ---")
  print(f"Chunk length: {len(chunks[i].page_content)}")
  print(f"Chunk metadata: {chunks[i].metadata}")
  print(chunks[i].page_content[:200] + "...") # Print first 200 characters of the chunk content

# Check the last few chunks (if there are more than 5)
if len(chunks) > 5:
    print(f"\nLast {min(5, len(chunks)-5)} chunks:")
    for i in range(max(0, len(chunks)-5), len(chunks)):
        print(f"--- Chunk {i+1} ---")
        print(f"Chunk length: {len(chunks[i].page_content)}")
        print(f"Chunk metadata: {chunks[i].metadata}")
        print(chunks[i].page_content[:200] + "...") # Print first 200 characters of the chunk content

# Additional checks
if len(chunks) > 0:
    # Check minimum chunk size
    min_len = min(len(chunk.page_content) for chunk in chunks)
    print(f"\nMinimum chunk length: {min_len}")
    if min_len < 50: # Based on min_chunk_size parameter
        print("Warning: Some chunks might be smaller than the specified minimum size.")

    # Check for empty chunks
    empty_chunks = sum(1 for chunk in chunks if len(chunk.page_content) == 0)
    print(f"Number of empty chunks: {empty_chunks}")

    # Check metadata presence (assuming add_metadata is True)
    metadata_missing = sum(1 for chunk in chunks if not hasattr(chunk, 'metadata') or not chunk.metadata)
    print(f"Number of chunks missing metadata: {metadata_missing}")
else:
    print("\nNo chunks were created.")


--- Chunking Validation ---
Number of documents loaded: 4114
Number of chunks created: 34743

First 5 chunks:
--- Chunk 1 ---
Chunk length: 125
Chunk metadata: {'producer': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'creator': 'Atop CHM to PDF Converter', 'creationdate': '2012-06-15T05:44:40+00:00', 'source': '/content/medical_diagnosis_manual.pdf', 'file_path': '/content/medical_diagnosis_manual.pdf', 'total_pages': 4114, 'format': 'PDF 1.7', 'title': 'The Merck Manual of Diagnosis & Therapy, 19th Edition', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-06-29T17:52:08+00:00', 'trapped': '', 'modDate': 'D:20250629175208Z', 'creationDate': 'D:20120615054440Z', 'page': 0}
anuguthalasandeepkumar@gmail.com
KUDXERABSF
personal use by anuguthalasandeepkum
shing the contents in part or full is liable...
--- Chunk 2 ---
Chunk length: 200
Chunk metadata: {'producer': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'creator': 'Atop CHM to PDF Converter', 'creationdate': '201

### Embedding

In [ ]:
# Note: For M4 with 16GB RAM, process chunks in small batches to avoid memory issues.
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Embed chunks in batches
embedded_chunks = []
batch_size = 100  # Safe for 16GB RAM
for i in range(0, len(chunks), batch_size):
    batch = chunks[i:i+batch_size]
    for j, chunk in enumerate(batch):
        try:
            embedding = embedding_function.embed_query(chunk.page_content)
            embedded_chunks.append((chunk, embedding))
        except Exception as e:
            print(f"Error embedding chunk {i+j}: {e}")
print(f"\n--- Embedding Validation ---")
print(f"Number of original chunks: {len(chunks)}")
print(f"Number of successfully embedded chunks: {len(embedded_chunks)}")
if len(embedded_chunks) == len(chunks):
    print("All chunks successfully embedded.")
else:
    print(f"Embedded {len(embedded_chunks)}/{len(chunks)} chunks.")

if len(embedded_chunks) > 0:
    first_embedded_chunk, first_embedding = embedded_chunks[0]
    print(f"Type of first embedding: {type(first_embedding)}")
    import numpy as np
    first_embedding_np = np.array(first_embedding)
    print(f"Dimension of first embedding: {len(first_embedding_np)}")
    print(f"First 10 values: {first_embedding_np[:10]}")

/tmp/ipython-input-23-3405431599.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


--- Embedding Validation ---
Number of original chunks: 34743
Number of successfully embedded chunks: 34743
All chunks successfully embedded.
Type of first embedding: <class 'list'>
Dimension of first embedding: 384
First 10 values: [-0.09975712  0.09495571 -0.00870426 -0.01502324  0.02987907  0.00238186
  0.09052261  0.07808921  0.02423851  0.00751337]


### Vector Database

In [ ]:
#Vector Database
import os, shutil

from langchain_community.vectorstores import Chroma

persist_directory = 'medical_db'
if os.path.exists(persist_directory):
    print(f"Removing existing database at {persist_directory}")
    shutil.rmtree(persist_directory, ignore_errors=True)

if not os.path.exists(persist_directory):
    os.makedirs(persist_directory)
    print(f"Created directory at {persist_directory}")

try:
    vector_db = Chroma.from_documents(
        documents=[chunk for chunk, embedding in embedded_chunks],
        embedding=embedding_function,
        persist_directory=persist_directory
    )
except Exception as e:
    print(f"Error creating Chroma database: {e}")
    raise

# Simplified vector database validation
print(f"\n--- Vector Database Validation ---")
if os.path.exists(persist_directory):
    print(f"Chroma database created at {persist_directory}.")
try:
    count = vector_db._collection.count()
    print(f"Number of items in database: {count}")
    print("Match with embedded chunks." if count == len(embedded_chunks) else "Item count mismatch.")
except Exception as e:
    print(f"Error retrieving database count: {e}")

Created directory at medical_db

--- Vector Database Validation ---
Chroma database created at medical_db.
Number of items in database: 34743
Match with embedded chunks.


### Retriever

In [ ]:
# prompt: code a retriever using the above code with the appropriate search method and k value
retriever = vector_db.as_retriever(
    search_type="similarity",  # Using similarity search
    search_kwargs={"k": 3}     # Retrieve top 3 similar documents
)

# --- Validation and Conclusions ---
print(f"\n--- Retriever Validation ---")
if retriever:
    print("Conclusion: Retriever successfully created.")
    try:
        # Test with queries from problem statement (e.g., sepsis, appendicitis, hair loss)
        sample_query = "What are the symptoms of appendicitis?"
        retrieved_docs = retriever.invoke(sample_query)
        print(f"\nRetrieved {len(retrieved_docs)} documents for a sample query.")
        if len(retrieved_docs) > 0:
            print("Sample of retrieved document content (first 100 chars):")
            print(retrieved_docs[0].page_content[:100] + "...")
        else:
            print("No documents retrieved. Check vector database content or query relevance.")

        print(f"\nRetriever configuration:")
        print(f"Search type: {retriever.search_type}")
        print(f"Search kwargs: {retriever.search_kwargs}")
        if retriever.search_kwargs.get("k") == 3:
            print("Conclusion: Retriever configured with correct k value (3).")
        else:
            print(f"Warning: Retriever's k value is {retriever.search_kwargs.get('k')}, expected 3.")
    except ValueError as ve:
        print(f"Database error during retrieval: {ve}")
    except Exception as e:
        print(f"Error during sample query with retriever: {e}")
        print("Conclusion: Retriever might not be configured correctly or database has issues.")
else:
    print("Conclusion: Failed to create the retriever.")


--- Retriever Validation ---
Conclusion: Retriever successfully created.

Retrieved 3 documents for a sample query.
Sample of retrieved document content (first 100 chars):
Symptoms and Signs
The classic symptoms of acute appendicitis are epigastric or periumbilical pain f...

Retriever configuration:
Search type: similarity
Search kwargs: {'k': 3}
Conclusion: Retriever configured with correct k value (3).


### System and User Prompt Template

In [ ]:
## 1. The system message describing the assistant's role.
## 2. A user message template including context and the question.

In [ ]:
# --- System and User Prompts ---
qna_system_message = "You are a knowledgeable medical assistant. Provide accurate, concise answers based solely on the provided context from the Merck Manuals. If the context is insufficient, state that you lack information."
qna_user_message_template = """Context: {context}\n\nQuestion: {question}\nAnswer concisely and factually."""


In [ ]:
# --- Evaluation Prompts ---
groundedness_rater_system_message = "You are an evaluator assessing the groundedness of a medical response. Rate the response based on whether all factual claims are supported by the provided context, on a scale of 1–5 (1: contradicts context, 5: fully supported). Return only the numeric rating."
relevance_rater_system_message = "You are an evaluator assessing the relevance of a medical response. Rate the response based on how directly it addresses the question, on a scale of 1–5 (1: irrelevant, 5: fully relevant). Return only the numeric rating."
user_message_template = """Question: {question}\nResponse: {answer}\nContext: {context}\nRating:"""


In [ ]:
# --- Queries to Test ---
queries_to_test = {
    "Query 1": "What is the protocol for managing sepsis in a critical care unit?",
    "Query 2": "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?",
    "Query 3": "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",
    "Query 4": "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",
    "Query 5": "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
}


In [ ]:
# --- RAG Response Function ---
def generate_rag_response(user_input, retriever, max_tokens=128, temperature=0, top_p=0.95, top_k=50):
    global qna_system_message, qna_user_message_template
    try:
        relevant_document_chunks = retriever.invoke(user_input)
        context_list = [d.page_content for d in relevant_document_chunks]
        context_for_query = ". ".join(context_list)[:4000]  # Limit for M4
        user_message = qna_user_message_template.replace('{context}', context_for_query).replace('{question}', user_input)
        prompt = f"{qna_system_message}\n{user_message}".strip()
        response = llm(
            prompt=prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k
        )
        return response['choices'][0]['text'].strip(), context_for_query
    except ValueError as ve:
        return f"Retrieval error: {ve}", ""
    except Exception as e:
        return f"Sorry, I encountered the following error: {e}", ""

In [ ]:
# --- Groundedness and Relevance Evaluation Function ---
def generate_ground_relevance_response(user_input, retriever, max_tokens=10, temperature=0, top_p=0.95, top_k=50):
    global groundedness_rater_system_message, relevance_rater_system_message, user_message_template
    try:
        # Generate RAG response and get context
        answer, context_for_query = generate_rag_response(user_input, retriever, max_tokens=128, temperature=0)

        # Groundedness evaluation
        groundedness_prompt = user_message_template.format(
            question=user_input,
            answer=answer,
            context=context_for_query
        )
        groundedness_response = llm(
            prompt=f"{groundedness_rater_system_message}\n{groundedness_prompt}",
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k
        )
        groundedness_rating = groundedness_response['choices'][0]['text'].strip()

        # Relevance evaluation
        relevance_prompt = user_message_template.format(
            question=user_input,
            answer=answer,
            context=context_for_query
        )
        relevance_response = llm(
            prompt=f"{relevance_rater_system_message}\n{relevance_prompt}",
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k
        )
        relevance_rating = relevance_response['choices'][0]['text'].strip()

        return groundedness_rating, relevance_rating, answer, context_for_query
    except Exception as e:
        return f"Error: {e}", f"Error: {e}", "", ""


In [ ]:
# --- Fine-tuning Parameters ---
param_combinations = [
    {"chunk_size": 500, "chunk_overlap": 100, "retriever_k": 3, "llm_max_tokens": 128, "llm_temperature": 0},
    {"chunk_size": 750, "chunk_overlap": 150, "retriever_k": 5, "llm_max_tokens": 200, "llm_temperature": 0.1},
    {"chunk_size": 1000, "chunk_overlap": 200, "retriever_k": 3, "llm_max_tokens": 128, "llm_temperature": 0},
    {"chunk_size": 500, "chunk_overlap": 100, "retriever_k": 5, "llm_max_tokens": 200, "llm_temperature": 0.2},
    {"chunk_size": 750, "chunk_overlap": 150, "retriever_k": 3, "llm_max_tokens": 128, "llm_temperature": 0.1}
]


In [ ]:
results = {}
evaluation_results = {}

print("\n--- Fine-tuning RAG Parameters ---")
for i, params in enumerate(param_combinations):
    print(f"\n--- Combination {i+1}: {params} ---")

    # Chunking (assumes get_data_chunks is defined as in previous code)
    current_chunks = get_data_chunks(
        documents,
        chunk_size=params["chunk_size"],
        chunk_overlap=params["chunk_overlap"],
        split_method="recursive",
        separators=["\n\n", "\n", ". ", " ", ""],
        min_chunk_size=50,
        add_metadata=True
    )
    print(f"  Created {len(current_chunks)} chunks.")

    # Embedding
    current_embedded_chunks = []
    batch_size = 50  # Safe for M4
    for j in range(0, len(current_chunks), batch_size):
        chunk_batch = current_chunks[j:j+batch_size]
        try:
            embeddings = embedding_function.embed_documents([chunk.page_content for chunk in chunk_batch])
            for chunk, embedding in zip(chunk_batch, embeddings):
                current_embedded_chunks.append((chunk, embedding))
        except Exception as e:
            print(f"  Error embedding batch {j}: {e}")
    print(f"  Embedded {len(current_embedded_chunks)} chunks.")

    # Vector Database
    persist_directory = f'medical_db_combo_{i+1}'
    if os.path.exists(persist_directory):
        shutil.rmtree(persist_directory, ignore_errors=True)
    os.makedirs(persist_directory, exist_ok=True)

    if current_embedded_chunks:
        try:
            current_vector_db = Chroma.from_documents(
                documents=[chunk for chunk, embedding in current_embedded_chunks],
                embedding=embedding_function,
                persist_directory=persist_directory
            )
            current_retriever = current_vector_db.as_retriever(
                search_type="similarity",
                search_kwargs={"k": params["retriever_k"]}
            )
            print(f"  Retriever created with k={params['retriever_k']}")

            # Query Execution and Evaluation
            results[f"Combination {i+1}"] = {}
            evaluation_results[f"Combination {i+1}"] = {}
            for query_name, query_text in queries_to_test.items():
                print(f"    Testing Query: {query_name}")
                groundedness_rating, relevance_rating, response_text, context = generate_ground_relevance_response(
                    query_text,
                    current_retriever
                )
                results[f"Combination {i+1}"][query_name] = response_text
                evaluation_results[f"Combination {i+1}"][query_name] = {
                    "groundedness_rating": groundedness_rating,
                    "relevance_rating": relevance_rating,
                    "context": context[:200] + "..." if context else "No context"
                }
                print(f"    Response: {response_text[:200]}...")
                print(f"    Groundedness Rating: {groundedness_rating}")
                print(f"    Relevance Rating: {relevance_rating}")
        except Exception as e:
            print(f"  Error creating vector DB or retriever: {e}")
    else:
        print("  No embedded chunks for vector DB.")



--- Fine-tuning RAG Parameters ---

--- Combination 1: {'chunk_size': 500, 'chunk_overlap': 100, 'retriever_k': 3, 'llm_max_tokens': 128, 'llm_temperature': 0} ---
  Created 34743 chunks.
  Embedded 34743 chunks.
  Retriever created with k=3
    Testing Query: Query 1


Llama.generate: 4 prefix-match hit, remaining 400 prompt tokens to eval
llama_perf_context_print:        load time =    1484.88 ms
llama_perf_context_print: prompt eval time =   29720.61 ms /   400 tokens (   74.30 ms per token,    13.46 tokens per second)
llama_perf_context_print:        eval time =   13778.35 ms /    84 runs   (  164.03 ms per token,     6.10 tokens per second)
llama_perf_context_print:       total time =   43544.44 ms /   484 tokens
Llama.generate: 3 prefix-match hit, remaining 502 prompt tokens to eval
llama_perf_context_print:        load time =    1484.88 ms
llama_perf_context_print: prompt eval time =   38130.68 ms /   502 tokens (   75.96 ms per token,    13.17 tokens per second)
llama_perf_context_print:        eval time =    1523.76 ms /     9 runs   (  169.31 ms per token,     5.91 tokens per second)
llama_perf_context_print:       total time =   39659.33 ms /   511 tokens
Llama.generate: 9 prefix-match hit, remaining 488 prompt tokens to eval
llama_perf_con

    Response: The protocol for managing sepsis in a critical care unit includes controlling hemorrhage, checking and providing respiratory assistance if necessary, keeping the patient warm, avoiding anything by mou...
    Groundedness Rating: 5. The response is fully supported by the
    Relevance Rating: 5
    Testing Query: Query 2


llama_perf_context_print:        load time =    1484.88 ms
llama_perf_context_print: prompt eval time =   35325.21 ms /   469 tokens (   75.32 ms per token,    13.28 tokens per second)
llama_perf_context_print:        eval time =   20861.03 ms /   127 runs   (  164.26 ms per token,     6.09 tokens per second)
llama_perf_context_print:       total time =   56256.94 ms /   596 tokens
Llama.generate: 3 prefix-match hit, remaining 614 prompt tokens to eval
llama_perf_context_print:        load time =    1484.88 ms
llama_perf_context_print: prompt eval time =   46081.01 ms /   614 tokens (   75.05 ms per token,    13.32 tokens per second)
llama_perf_context_print:        eval time =    1461.31 ms /     9 runs   (  162.37 ms per token,     6.16 tokens per second)
llama_perf_context_print:       total time =   47547.13 ms /   623 tokens
Llama.generate: 9 prefix-match hit, remaining 600 prompt tokens to eval
llama_perf_context_print:        load time =    1484.88 ms
llama_perf_context_print: p

    Response: The common symptoms for appendicitis include epigastric or periumbilical pain followed by brief nausea, vomiting, and anorexia, which later shifts to the right lower quadrant. The pain increases with ...
    Groundedness Rating: 5. The response accurately describes the common symptoms
    Relevance Rating: 5
    Testing Query: Query 3


...

    Response: Based on the context provided, the possible causes for sudden patchy hair loss could be alopecia areata. Effective treatments for alopecia areata include topical, intralesional, or systemic corticoste...
    Groundedness Rating: 5. The response accurately identifies alo
    Relevance Rating: 5
    Testing Query: Query 4


llama_perf_context_print:        load time =    1484.88 ms
llama_perf_context_print: prompt eval time =   45964.15 ms /   611 tokens (   75.23 ms per token,    13.29 tokens per second)
llama_perf_context_print:        eval time =   20810.91 ms /   127 runs   (  163.87 ms per token,     6.10 tokens per second)
llama_perf_context_print:       total time =   66846.59 ms /   738 tokens
Llama.generate: 3 prefix-match hit, remaining 756 prompt tokens to eval
llama_perf_context_print:        load time =    1484.88 ms
llama_perf_context_print: prompt eval time =   56608.90 ms /   756 tokens (   74.88 ms per token,    13.35 tokens per second)
llama_perf_context_print:        eval time =    1536.95 ms /     9 runs   (  170.77 ms per token,     5.86 tokens per second)
llama_perf_context_print:       total time =   58151.14 ms /   765 tokens
Llama.generate: 9 prefix-match hit, remaining 742 prompt tokens to eval
llama_perf_context_print:        load time =    1484.88 ms
llama_perf_context_print: p

    Response: Do not use contractions or jargon.

Answer: For a person with a brain injury resulting in neurologic deficits, rehabilitation is necessary. This includes physical, occupational, and speech therapy, sk...
    Groundedness Rating: 5

The response is fully supported by
    Relevance Rating: 5
    Testing Query: Query 5


llama_perf_context_print:        load time =    1484.88 ms
llama_perf_context_print: prompt eval time =   49344.69 ms /   647 tokens (   76.27 ms per token,    13.11 tokens per second)
llama_perf_context_print:        eval time =   20857.06 ms /   127 runs   (  164.23 ms per token,     6.09 tokens per second)
llama_perf_context_print:       total time =   70274.85 ms /   774 tokens
Llama.generate: 3 prefix-match hit, remaining 792 prompt tokens to eval
llama_perf_context_print:        load time =    1484.88 ms
llama_perf_context_print: prompt eval time =   60946.54 ms /   792 tokens (   76.95 ms per token,    12.99 tokens per second)
llama_perf_context_print:        eval time =    1479.79 ms /     9 runs   (  164.42 ms per token,     6.08 tokens per second)
llama_perf_context_print:       total time =   62431.31 ms /   801 tokens
Llama.generate: 9 prefix-match hit, remaining 778 prompt tokens to eval
llama_perf_context_print:        load time =    1484.88 ms
llama_perf_context_print: p

    Response: Do not add personal opinions or assumptions.

The person with a fractured leg should receive initial treatment for any life-threatening injuries, such as hemorrhagic shock. For the fracture itself, th...
    Groundedness Rating: 5
The response is fully supported by the
    Relevance Rating: 5
The response directly addresses the question by


Response Quality: RAG-based answers are highly accurate, context-specific, and consistently grounded in the provided medical manual (e.g., Merck Manual).

Groundedness and Relevance:

Answers are directly supported by retrieved context, minimizing hallucination and ensuring factual correctness.

The system can handle complex, multi-part queries with detailed, stepwise protocols and explanations.

Transparency: The RAG approach enables traceability, as each answer is based on retrieved, authoritative content. This is critical for clinical decision support and regulatory compliance.

Consistency: Responses are uniform in style and depth, and ambiguity is minimized. The system can explicitly state when the context is insufficient to answer a question.

Limitations:

The quality of the answer depends on the quality and coverage of the underlying knowledge base. If the manual lacks information on a rare condition, the answer may be incomplete.

Retrieval and generation can be slower than direct LLM responses, especially for large document sets.

In [ ]:
# --- Compare Results ---
print("\n\n--- Comparison of Results ---")
for query_name, query_text in queries_to_test.items():
    print(f"\n### {query_name}: {query_text}")
    for combo_name, query_results in results.items():
        response_text = query_results.get(query_name, "Response not found")
        eval_data = evaluation_results.get(combo_name, {}).get(query_name, {})
        print(f"\n#### {combo_name} (Chunk Size: {param_combinations[int(combo_name.split(' ')[1])-1]['chunk_size']}, "
              f"Overlap: {param_combinations[int(combo_name.split(' ')[1])-1]['chunk_overlap']}, "
              f"Retriever k: {param_combinations[int(combo_name.split(' ')[1])-1]['retriever_k']}, "
              f"LLM Tokens: {param_combinations[int(combo_name.split(' ')[1])-1]['llm_max_tokens']}, "
              f"LLM Temp: {param_combinations[int(combo_name.split(' ')[1])-1]['llm_temperature']})")
        print(f"Response: {response_text}")
        print(f"Groundedness Rating: {eval_data.get('groundedness_rating', 'N/A')}")
        print(f"Relevance Rating: {eval_data.get('relevance_rating', 'N/A')}")
        print(f"Context Preview: {eval_data.get('context', 'N/A')}")
        print("-" * 50)




--- Comparison of Results ---

### Query 1: What is the protocol for managing sepsis in a critical care unit?

#### Combination 1 (Chunk Size: 500, Overlap: 100, Retriever k: 3, LLM Tokens: 128, LLM Temp: 0)
Response: The protocol for managing sepsis in a critical care unit includes controlling hemorrhage, checking and providing respiratory assistance if necessary, keeping the patient warm, avoiding anything by mouth, draining abscesses, and surgically excising necrotic tissues. Septic foci must be eliminated to prevent further deterioration. Normalization of blood glucose also improves outcome.
Groundedness Rating: 5. The response is fully supported by the
Relevance Rating: 5
Context Preview: 16 - Critical Care Medicine
Chapter 222. Approach to the Critically Ill Patient
Introduction
Critical care medicine specializes in caring for the most seriously ill patients. These patients are best
t...
--------------------------------------------------

#### Combination 2 (Chunk Size: 750, Ov

In [ ]:
# --- Evaluation Summary ---
print("\n--- Evaluation Results Summary ---")
eval_summary = {}
for combo_name in evaluation_results:
    eval_summary[combo_name] = {}
    for query_name in queries_to_test:
        eval_data = evaluation_results[combo_name].get(query_name, {})
        eval_summary[combo_name][query_name] = {
            "Groundedness": eval_data.get("groundedness_rating", "N/A"),
            "Relevance": eval_data.get("relevance_rating", "N/A")
        }
eval_df = pd.DataFrame.from_dict({(c, q): eval_summary[c][q] for c in eval_summary for q in eval_summary[c]}, orient='index')
print(eval_df)


--- Evaluation Results Summary ---
                                                            Groundedness  \
Combination 1 Query 1          5. The response is fully supported by the   
              Query 2  5. The response accurately describes the commo...   
              Query 3          5\nThe response accurately identifies alo   
              Query 4   5. The response accurately reflects the context,   
              Query 5  5. The response accurately reflects the contex...   
Combination 2 Query 1  5. The response accurately describes the proto...   
              Query 2          5\nAll factual claims are fully supported   
              Query 3          5. All factual claims are fully supported   
              Query 4   5. The response accurately reflects the context,   
              Query 5          5\nAll factual claims are fully supported   
Combination 3 Query 1          7\nThe response is fully supported by the   
              Query 2          5. All factual claims

**Actionable Insights and Recommendations**

**Key Business Insights**

1.Implementing a Retrieval-Augmented Generation (RAG) system using trusted medical manuals (e.g., Merck Manual) enables healthcare professionals to access accurate, up-to-date information rapidly, reducing time spent searching for answers and minimizing information overload.

2.Automated, context-aware responses to common clinical questions (diagnosis, treatment, drug information) streamline decision-making, especially in high-pressure environments like critical care.

3.Centralizing medical knowledge and protocols through AI ensures consistent application of best practices across the organization, reducing variability in care and supporting evidence-based medicine.

4.The system’s ability to provide grounded, relevant, and context-specific answers supports safer, more reliable patient management.

5.The RAG-based solution is scalable: as new medical knowledge emerges, the system can be updated without retraining the core AI, ensuring ongoing relevance and compliance.

6.By automating routine queries and triage, clinical staff can focus on complex cases, improving workforce productivity and reducing burnout.

7.Usage analytics from the AI system can identify knowledge gaps, frequently asked questions, and areas where additional staff training or protocol updates are needed.

8.Continuous evaluation of AI responses (groundedness and relevance ratings) provides a feedback loop for system improvement and regulatory compliance.

**Recommendations**

1.Deploy the AI assistant at key points of care (e.g., nurse stations,
emergency rooms, telemedicine platforms) to support real-time clinical decision-making.

2.Ensure seamless interoperability with existing electronic health record (EHR) systems for context-aware recommendations.

3.Start with high-frequency, high-risk clinical scenarios (e.g., sepsis management, acute abdominal pain, trauma protocols) to maximize immediate value and demonstrate ROI.

4.Expand coverage to specialty knowledge and rare conditions as the system matures.

5.Provide targeted training for clinicians to build trust in AI recommendations and clarify the system’s role as a decision-support tool, not a replacement for clinical judgment.

6.Establish clear escalation protocols for ambiguous or unsupported queries.

7.Regularly review system performance using groundedness and relevance metrics to ensure accuracy and clinical safety.

8.Use analytics to refine knowledge bases, update protocols, and identify opportunities for further automation.

9.Ensure all patient data and medical content are handled in compliance with healthcare regulations (e.g., HIPAA).

10.Maintain transparency in AI decision-making to support auditability and regulatory review.





